In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import hydra

In [26]:
a = 1

In [27]:
directory = "h:/test_2005/"

trading_dates = pd.read_csv(directory + "trading_dates.csv", index_col=0)
trading_dates["MarketDate"] = pd.to_datetime(trading_dates["MarketDate"])

data = pd.read_csv(directory+"rolling_universe.csv", index_col=0)["Infocode"]
data.index = pd.to_datetime(data.index)
rolling_universe = {}
for key in data.index:
    rolling_universe[key] = np.array(data[key].split()).astype('int')
    
infocode_dict = pd.read_csv(directory + "infocode_dict.csv", index_col=0)["Ticker"].to_dict()
infocode_to_trid = pd.read_csv(directory + "infocode_to_trid.csv", index_col=0)["ID"].to_dict()
ticker_dict = pd.read_csv(directory + "ticker_dict.csv", index_col=0)["Infocode"].to_dict()
infocode_sector = pd.read_csv(directory + "infocode_sector.csv", index_col=0)["ECONSECT"].to_dict()

In [28]:
def read_stock_data(directory, name):
    stock_data = pd.read_csv(directory + name, index_col=0)
    stock_data.index = pd.to_datetime(stock_data.index)
    stock_data.columns = stock_data.columns.astype("int")
    return stock_data

In [29]:
stock_prices = read_stock_data(directory, 'stock_prices.csv')
stock_iv3m = read_stock_data(directory, 'stock_iv3m.csv')
stock_iv6m = read_stock_data(directory, 'stock_iv6m.csv')
stock_iv12m = read_stock_data(directory, 'stock_iv12m.csv')
stock_iv2m = read_stock_data(directory, 'stock_iv2m.csv')
stock_iv1m = read_stock_data(directory, 'stock_iv1m.csv')
stock_iv1m50delta = read_stock_data(directory, 'stock_iv1m50delta.csv')
stock_iv1m25delta = read_stock_data(directory, 'stock_iv1m25delta.csv')
stock_iv1m75delta = read_stock_data(directory, 'stock_iv1m75delta.csv')
stock_iv2m50delta = read_stock_data(directory, 'stock_iv2m50delta.csv')
stock_iv2m25delta = read_stock_data(directory, 'stock_iv2m25delta.csv')
stock_iv2m75delta = read_stock_data(directory, 'stock_iv2m75delta.csv')

In [30]:
stock_asset_per_equity = read_stock_data(directory, 'stock_asset_per_equity.csv')
stock_asset_per_marketcap = read_stock_data(directory, 'stock_asset_per_marketcap.csv')
stock_eps = read_stock_data(directory, 'stock_eps.csv')
stock_roe = read_stock_data(directory, 'stock_roe.csv')
stock_cps = read_stock_data(directory, 'stock_cps.csv')
stock_cpx_per_marketcap = read_stock_data(directory, 'stock_cpx_per_marketcap.csv')
stock_dps = read_stock_data(directory, 'stock_dps.csv')
stock_marketcap = read_stock_data(directory, 'stock_marketcap.csv')

In [31]:
trading_interval = 5
holding_period = 63

start_date = pd.to_datetime('20100101')
end_date = trading_dates["MarketDate"].iloc[-1]

rebalance_dates = []
valid_dates = trading_dates[(trading_dates["MarketDate"] >= start_date) & (trading_dates["MarketDate"] <= end_date)]
for i in range(0, len(valid_dates), trading_interval):
    rebalance_dates.append(valid_dates["MarketDate"].iloc[i])
    
trading_dates = list(pd.to_datetime(trading_dates["MarketDate"].values))

In [32]:
stock_returns = np.log(stock_prices/stock_prices.shift())

In [33]:
stock_rv1m = pd.rolling_std(stock_returns, 21) * np.sqrt(252)
stock_rv2m = pd.rolling_std(stock_returns, 42) * np.sqrt(252)
stock_rv3m = pd.rolling_std(stock_returns, 63) * np.sqrt(252)
stock_rv6m = pd.rolling_std(stock_returns, 126) * np.sqrt(252)
stock_rv12m = pd.rolling_std(stock_returns, 252) * np.sqrt(252)
stock_rv10d = pd.rolling_std(stock_returns, 10) * np.sqrt(252)

In [34]:
vix = pd.read_csv(directory + 'vix.csv', index_col=0)
vix = pd.Series(vix['vix'].values, index=pd.to_datetime(vix.index))
vix_ma = pd.rolling_mean(vix, 10)

In [35]:
spx = pd.read_csv(directory + 'spx.csv', index_col=0)
spx = pd.Series(spx['spx'].values, index=pd.to_datetime(spx.index))

In [36]:
rolling_universe_small = {}
rolling_universe_mid = {}
rolling_universe_large = {}
for date in trading_dates:
    MC = stock_marketcap.loc[date][rolling_universe[date]]
    rolling_universe_small[date] = np.array(MC[MC<10000].index)
    rolling_universe_mid[date] = np.array(MC[(MC<50000) & (MC>10000)].index)
    rolling_universe_large[date] = np.array(MC[MC>50000].index)

In [37]:
infocode_sectors = pd.Series(infocode_sector)
sectors = ['Financials', 'Consumer Cyclicals', 'Industrials', 'Technology', 'Healthcare', 'Consumer Non-Cyclicals', 'Energy']
rolling_universe_sectors = {}
for sector in sectors:
    rolling_universe_sectors[sector] = {}
for date in trading_dates:
    universe = infocode_sectors[rolling_universe[date]]
    for sector in sectors:
        if sector != 'Others':
            rolling_universe_sectors[sector][date] = np.array(universe[universe == sector].index)
        else:
            rolling_universe_sectors[sector][date] = np.array(universe[(universe == 'Utilities') | (universe == 'Basic Materials') | (universe == 'Telecommunications Services') | (universe == 'Others')].index)

In [ ]:
hydraDB = "equities_lqr_dev"
hydraPath = "/eq/lqr/data/df_test"


hydra.db = hydra.dbopen(hydraDB)
loader = hydra.db.read_or_new( 'Container', '%s' % hydraPath )
loader.contents = df
loader.setEntitlement('Equities LQR')
loader.write( overwrite = True )

# Write to and Read from Hydra

## Write to Hydra

In [38]:
def write_to_hydra(df, name, hydraDB="equities_lqr_dev", hydraPath="/eq/lqr/data/"):
    hydra.db = hydra.dbopen(hydraDB)
    loader = hydra.db.read_or_new( 'Container', '%s' % hydraPath + name )
    loader.contents = df
    loader.setEntitlement('Equities LQR')
    loader.write( overwrite = True )

In [39]:
write_to_hydra(trading_dates, 'trading_dates')
write_to_hydra(rolling_universe, 'rolling_universe')
write_to_hydra(infocode_dict, 'infocode_dict')
write_to_hydra(infocode_to_trid, 'infocode_to_trid')
write_to_hydra(infocode_sector, 'infocode_sector')
write_to_hydra(ticker_dict, 'ticker_dict')

write_to_hydra(stock_prices, 'stock_prices')
write_to_hydra(stock_iv3m, 'stock_iv3m')
write_to_hydra(stock_iv6m, 'stock_iv6m')
write_to_hydra(stock_iv12m, 'stock_iv12m')
write_to_hydra(stock_iv1m, 'stock_iv1m')
write_to_hydra(stock_iv2m, 'stock_iv2m')
write_to_hydra(stock_iv1m50delta, 'stock_iv1m50delta')
write_to_hydra(stock_iv1m25delta, 'stock_iv1m25delta')
write_to_hydra(stock_iv1m75delta, 'stock_iv1m75delta')
write_to_hydra(stock_iv2m50delta, 'stock_iv2m50delta')
write_to_hydra(stock_iv2m25delta, 'stock_iv2m25delta')
write_to_hydra(stock_iv2m75delta, 'stock_iv2m75delta')

write_to_hydra(stock_asset_per_equity, 'stock_asset_per_equity')
write_to_hydra(stock_asset_per_marketcap, 'stock_asset_per_marketcap')
write_to_hydra(stock_eps, 'stock_eps')
write_to_hydra(stock_roe, 'stock_roe')
write_to_hydra(stock_cps, 'stock_cps')
write_to_hydra(stock_cpx_per_marketcap, 'stock_cpx_per_marketcap')
write_to_hydra(stock_dps, 'stock_dps')
write_to_hydra(stock_marketcap, 'stock_marketcap')

write_to_hydra(spx, 'spx')
write_to_hydra(vix, 'vix')

## Read from Hydra

In [40]:
def read_from_hydra(name, hydraDB="equities_lqr_dev", hydraPath="/eq/lqr/data/"):
    return hydra.dbopen(hydraDB).read(hydraPath + name).contents

In [41]:
trading_dates = read_from_hydra('trading_dates')
rolling_universe = read_from_hydra('rolling_universe')
infocode_dict = read_from_hydra('infocode_dict')
infocode_to_trid = read_from_hydra('infocode_to_trid')
infocode_sector = read_from_hydra('infocode_sector')
ticker_dict = read_from_hydra('ticker_dict')

stock_prices = read_from_hydra('stock_prices')
stock_iv1m = read_from_hydra('stock_iv1m')
stock_iv2m = read_from_hydra('stock_iv2m')
stock_iv3m = read_from_hydra('stock_iv3m')
stock_iv6m = read_from_hydra('stock_iv6m')
stock_iv12m = read_from_hydra('stock_iv12m')
stock_iv1m50delta = read_from_hydra('stock_iv1m50delta')
stock_iv1m25delta = read_from_hydra('stock_iv1m25delta')
stock_iv1m75delta = read_from_hydra('stock_iv1m75delta')
stock_iv2m50delta = read_from_hydra('stock_iv2m50delta')
stock_iv2m25delta = read_from_hydra('stock_iv2m25delta')
stock_iv2m75delta = read_from_hydra('stock_iv2m75delta')

stock_asset_per_equity = read_from_hydra('stock_asset_per_equity')
stock_asset_per_marketcap = read_from_hydra('stock_asset_per_marketcap')
stock_eps = read_from_hydra('stock_eps')
stock_roe = read_from_hydra('stock_roe')
stock_cps = read_from_hydra('stock_cps')
stock_cpx_per_marketcap = read_from_hydra('stock_cpx_per_marketcap')
stock_dps = read_from_hydra('stock_dps')
stock_marketcap = read_from_hydra('stock_marketcap')

spx = read_from_hydra('spx')
vix = read_from_hydra('vix')

In [42]:
trading_interval = 5
holding_period = 63
start_date = pd.to_datetime('20100101')
end_date = trading_dates[-1]

rebalance_dates = []
valid_dates = np.array(trading_dates)[(np.array(trading_dates) > start_date) & (np.array(trading_dates) < end_date)]
for i in range(0, len(valid_dates), trading_interval):
    rebalance_dates.append(valid_dates[i])

In [43]:
stock_returns = np.log(stock_prices/stock_prices.shift())

stock_rv10d = pd.rolling_std(stock_returns, 10) * np.sqrt(252)
stock_rv1m = pd.rolling_std(stock_returns, 21) * np.sqrt(252)
stock_rv2m = pd.rolling_std(stock_returns, 42) * np.sqrt(252)
stock_rv3m = pd.rolling_std(stock_returns, 63) * np.sqrt(252)
stock_rv6m = pd.rolling_std(stock_returns, 126) * np.sqrt(252)
stock_rv12m = pd.rolling_std(stock_returns, 252) * np.sqrt(252)

In [44]:
infocode_sectors = pd.Series(infocode_sector)
sectors = ['Financials', 'Consumer Cyclicals', 'Industrials', 'Technology', 'Healthcare', 'Consumer Non-Cyclicals', 'Energy']
rolling_universe_sectors = {}
for sector in sectors:
    rolling_universe_sectors[sector] = {}
for date in trading_dates:
    universe = infocode_sectors[rolling_universe[date]]
    for sector in sectors:
        if sector != 'Others':
            rolling_universe_sectors[sector][date] = np.array(universe[universe == sector].index)
        else:
            rolling_universe_sectors[sector][date] = np.array(universe[(universe == 'Utilities') | (universe == 'Basic Materials') | (universe == 'Telecommunications Services') | (universe == 'Others')].index)